# STOR609 Assessment 3 
## Replication and Reproduction Study
Group 3: Vlad Bercovici, Malcolm Connolly, Rebekah Fearnhead, Niharika Peddinenikalva


## 1 Introduction

Pig is a turn based jeopardy dice game over a number of rounds. Players may roll any number of times, scoring the cumulative sum of the numbers rolled, with the jeopardy condition that if a player rolls a $1$ then their round score is zero. The game continues until either player attains a score of at least $100$.

The aim of this package and notebook is to constitute a replication study of the findings on the optimal play of the game Pig published in the journal UMAP (Neller and Presser, 2004). That is, to provide substantial evidence for replication of the original results in the article. Neller and Presser claim to apply Value Iteration to the game of Pig, and produce several figures illustrating an optimal policy for playing the game. 

We reproduce all figures relating to the optimal policy in the original article, rigorously describe the methodology from which they are derived, and provide our original source code as an open source package and repository.



## 2 Methodology

In this section we give a detailed account of our notation and application of the Value Iteration formalism to reproduce the results of Neller and Pressers article. We also describe instances where these were different from those described in the paper. 

### 2.1 Piglet

The game of Piglet is a simplification of the game of Pig discussed in the paper involving flipping a coin instead of rolling a die, where the coin landing on heads or tails would score $1$ and $0$ respectively. This game is only studied in the case when the goal or winning condition is a score of $2$. 

#### 2.1.1 States
Following the paper, we can define the states as $3$-tuples, $(i, \ j, \ k)$, where $i$ is the current score, $j$ is the current score of the opponent and $k$ is the current turn score of the player. Neller and Presser state 'winning and losing states are terminal', however by their own reasoning utilising the symmetry, we need only consider consider a single further 'winning' state $(W, \ L, \ 0)$. The states are then,

$$S = \{ (i, \ j, \ k) : i, \ j, \ k \in \{0,1\}, \  i+k <2 \} \cup \{(W, \ L, \ 0)\}.$$

#### 2.1.2 Actions

The actions $A$ in the game of Piglet are whether to flip or not, $A=\{\text{flip},\text{hold}\}$.


#### 2.1.3 Transitions

There are three types of transitions that we consider.

##### (i)
When the action is to **flip** and the outcome is **heads** with probability $\frac{1}{2}$, and the transition is given by:
$$(i, \ j, \ k) \  \mapsto \ \begin{cases} (i, \ j, \ k+1) &\text{ if }k+1<2 \\
(W, \ L, \ 0) &\text{ otherwise.}\end{cases}$$

##### (ii)
When the action is to **flip** and the outcome is **tails** with probability $\frac{1}{2}$, and the transition is given by:
$$(i, \ j, \ k) \mapsto (j, \  i, \ 0). $$

##### (iii)

Finally, if the action is to **hold** then with probability $1$ the transition is given by:
$$(i, \  j,  \ k) \mapsto (j, \ i+k, \ 0).$$

Transitions (ii) and (iii) utilise symmetry. Our probability of winning the game is the probability that the opponent does not win after having lost, or having held on the previous turn. However, as the opponent is also assumed to be playing optimally, we need only consider winning in this opposing state from our point of view, and existing list of states, as if we were the opponent. 


#### 2.1.4 Rewards

We award a reward of $1$ for any transition from a non-winning state to the winning state, and $0$ otherwise. 

$$R_{ss'}^{a} = \begin{cases} 1 &\text{ if }s=(i, \ j, \ k), \ s'=(W, \ L, \ 0), \text{ where } a=\text{flip} \text { and }i+k<2  \\
                         0 &\text{ otherwise.}\end{cases}$$

#### 2.1.5 Iteration

As in the paper, we conflate probability of winning from a given state and the value of that state, taking $P_{(i, \ j, \ k)} = V( (i, \ j, \ k) ).$ In our implementation we directly solve the $6$ equations for $P_{s}$ with $s\in S\setminus \{(W,\ L, \ 0)\}$.

$$P_{(i, \ j, \ k)} = \max \left\{ \overbrace{\frac{1}{2}\left[\left( 1-{P}_{(j, \ i, \ 0)} \right) + P_{(i, \ j, \ k+1)}\right]}^{a = \text{flip}}, \overbrace{1- P_{(j, \ i+k, \ 0)}}^{a= \text{hold}} \right\}. $$

Taking $P_{(1,0,1)}=P_{(1,1,1)}=1$ if the flip increments the player score. These are equivalent to equations (1) in the paper (pg 28, Neller and Presser, 2004).

Initially we took the values $P_{s} = 0, \forall s\in S\setminus \{(W,\ L, \ 0)\}$, and $P_{(W,L,0)} = 1$. Initial iterations were tabulated by hand, and are displayed here for completeness.





| Iteration |$P_{(0,0,0)}$ | $P_{(0,0,1)}$ | $P_{(0,1,0)}$ | $P_{(0,1,1)}$ | $P_{(1,0,0)}$ | $P_{(1,1,0)}$ |
| --- | --- | --- | --- | --- | --- | --- |
| 1 | $\max\left\{\frac{1}{2}, 1\right\} = 1$ | $\max\left\{1, 1\right\} = 1$ | $\max\left\{\frac{1}{2}, 1\right\} = 1$ | $\max\left\{1, 1\right\} = 1$ | $\max\left\{1, 1\right\} = 1$ | $\max\left\{1, 1\right\} = 1$ |
| 2 | $\max\left\{\frac{1}{2}, 0\right\} = \frac{1}{2}$ |$\max\left\{\frac{1}{2}, 0\right\} = \frac{1}{2}$ | $\max\left\{\frac{1}{2}, 0\right\} = \frac{1}{2}$ | $\max\left\{\frac{1}{2}, 0\right\} = \frac{1}{2}$ | $\max\left\{\frac{1}{2}, 0\right\} = \frac{1}{2}$ | $\max\left\{\frac{1}{2}, 0\right\} = \frac{1}{2}$ |
| 3 | $\max\left\{\frac{1}{2}, \frac{1}{2}\right\} = \frac{1}{2}$ |$\max\left\{\frac{3}{4}, \frac{1}{2}\right\} = \frac{3}{4}$ | $\max\left\{\frac{1}{2}, \frac{1}{2}\right\} = \frac{1}{2}$ | $\max\left\{\frac{3}{4}, \frac{1}{2}\right\} = \frac{3}{4}$ | $\max\left\{\frac{3}{4}, \frac{1}{2}\right\} = \frac{3}{4}$ | $\max\left\{\frac{3}{4}, \frac{1}{2}\right\} = \frac{3}{4}$ |

### 2.2 Pig

Our methodology for finding the optimal policy for the game of pig is similar to that of piglet in section 2.1.

#### 2.2.1 States

Again we can define the states as $3$-tuples, $(i, \ j, \ k)$, where $i$ is the current score, $j$ is the current score of the opponent and $k$ is the current turn score of the player, in addition to two further 'winning' and 'losing' states $(W, \ L, \ 0)$ and $(L, \ W, \ 0)$. We include the extra state for completeness, though it is not essential to obtain the results. The states $S$ are given by,

$$S = \{ (i, \ j, \ k) : i, \ j, \ k \in \{0,1\}, \  i+k < 100 \} \cup \{(W, \ L, \ 0)\}\cup \{(L, \ W, \ 0)\}.$$

#### 2.2.2 Actions

The actions $A$ in the game of pig answer the question of whether to roll or not to roll. That is $A=\left\{ \text{roll}, \ \text{hold} \right\}.$

#### 2.2.3 Transitions

There are three types of transitions that we consider.

##### (i)
When the action is to **roll**, the outcome is $r = \{2,3,\ldots\}$ with probability $\frac{1}{6}$, and the transition is given by:
$$(i, \ j, \ k) \  \mapsto \ \begin{cases} (i, \ j, \ k+r) &\text{ if }k+r<100 \\
(W, \ L, \ 0) &\text{ otherwise.}\end{cases}$$

##### (ii)
When the action is to **roll**, the outcome is $r=1$ with probability $\frac{1}{6}$, and the transition is given by:
$$(i, \ j, \ k) \mapsto (j, \  i, \ 0). $$

##### (iii)

Finally, if the action is to **hold** then with probability $1$ the transition is given by:
$$(i, \  j,  \ k) \mapsto (j, \ i+k, \ 0).$$


#### 2.2.4 Rewards

We award a reward of $1$ for any transition from a non-winning state to the winning state, and $0$ otherwise. 

$$R_{ss'}^{a} = \begin{cases} 1 &\text{ if }s=(i, \ j, \ k), \ s'=(W, \ L, \ 0), \text{ where } a=\text{roll,} \text { and }i+k<100  \\
                         0 &\text{ otherwise.}\end{cases}$$

#### 2.2.5 Iteration

Again we take $P_{(i, \ j, \ k)} = V( (i, \ j, \ k) ).$ In our implementation we directly solve the $6$ equations for $P_{s}$ with $s\in S\setminus \{(W,\ L, \ 0)\}$.

$$P_{(i, \ j, \ k)} = \max \left\{ \overbrace{\frac{1}{6}\left[\left( 1-{P}_{(j, \ i, \ 0)} \right) + P_{(i, \ j, \ k+2)}+ P_{(i, \ j, \ k+3)}+ \ldots + P_{(i, \ j, \ k+6)}\right]}^{a = \text{roll}}, \overbrace{1- P_{(j, \ i+k, \ 0)}}^{a= \text{hold}} \right\}. $$

The authors claim this yields $505,000$ equations. We calculate this number using inclusion-exclusion principle. Counting all possible triples $(i,j,k)$, where each can take a value from $0$ to $99$, gives $100^3$ triples. Now considering the triples $(i,j,k)$ where $i+k\geq 100$, we count the choices of $k$ that any value of $i$ affords according to the table below.

| $i$ | $k\geq 100 - i$ | choices for $k$ |
| --- | --- | --- | 
| $i=1$ | $k\geq 99$ | $1$ | 
| $i=2$ | $k\geq 98$ | $2$ |
| $\vdots$ | $\vdots$ | $\vdots$ |
| $i=99$ | $k\geq 1$ | $99$ |

The number of choices for $k$ is then the sum of the last column $1+2+\cdots+99 = \frac{99\times 100}{2}$. Further in each case there is a choice of $j$, and so the total number of equations is,

$$100^3 - \frac{99\times 100}{2}\times 100 = 505,000. $$

## 3 Results

In this section we give an account of each of the figures we were able to replicate from the paper, thereby corroborating the main results of the paper regarding optimal play. 

### 3.1 Optimal piglet play (Figure 2)

<!-- Insert fig 2 -->

Figure 2 in the paper shows the results of applying value iteration to the game of piglet where the goal is to achieve $2$ points. Each of the lines represents how the estimates for each of the win probabilities changes as iterations of the algorithm are performed. The graph in the paper shows, for each of the states, the win probability converges within $25$ iterations, and the values to which they converge are the same as the exact values given earlier in the paper.

By running our code, we get similar results, with the values that the win probabilities converge to after $25$ iterations being the same as in the paper. From iterations $5$ to $25$, the shape of lines produced showing the behaviour of the estimates are similar to those in the paper, with the estimates for $P_{(0,0,1)}$, $P_{(1,1,0)}$ and $P_{(0,0,0)}$ taking longer to converge than those of $P_{(1,0,0)}$, $P_{(0,1,1)}$ and $P_{(1,0,0)}$ which converge more smoothly. 

### 3.2 Optimal pig play surface (Figure 3)

<!-- Insert fig 3 -->

Figure $3$ looks at the game of pig and shows the optimal policy for each of the states for which $i,j$ and $k$ are coordinate axes. In the graph the boundary surface is shown between the states where rolling again is the optimal policy and those where the optimal policy is to hold.

We have replicated this by taking the optimal policy we computed through value iteration for each possible state and plotted all the points at which it would be optimal to roll again.

Comparing our graph to the one in the paper shows that the general behaviour of the policy is the same for both our implementations. However there are some difficulties in the comparison as we are only given two views of the surface. We are also not told the states representing the points on the boundary so it is not possible to check that we have the same optimal policy for all of the states. If either the method of Neller and Presser or our method for value iteration has not fully converged, the slight differences could lead to some of the states having different 'optimal' policies if the optimal values for the two possible actions are almost equal.

We have provided our converged policy for future comparison and examination in a pkl file of our repository. 

### 3.3 Reachable states (Figures 4, 5 and 6)

<!-- insert fig 4 -->

Figure 4 takes a cross section of the surface produced in Figure 3 and compares it to the states that are reachable if the optimal policy is used (with player 2 using any policy). Figure $4$ is easier to compare to the graph created with our results, as it is only in two dimensions. The shape of both the cross section of the reachable region and the optimal policy boundary look the same in both graphs. From the graphs it can be seen how the optimal policy boundary compares to the 'hold at 20' boundary. For lower values of player 1's score they want to roll while they have a turn total of up to $23$. This value decreases until a player score of around $75$ at which point they wish to roll until they have won. The cross section of the reachable area can exceed the optimal policy boundary by up to $6$, which is caused by the maximum score that can be rolled on the boundary being $6$.

<!-- insert fig 5 -->

Figure 5 is then used to show all the states that can be reached with a player following the optimal policy. 

Comparing the results of this to figure 5 in the paper produces a similar surface showing the same behaviour. For example, there are $4$ gaps in the reachable states. As mentioned in the paper, an optimal player with score $0$ will never hold before reaching a turn total less than $21$ which can be seen in the graph we have produced. There are also $3$ other gaps where the states are not reachable under the optimal policy, however as the current score of player 2 increases, these states become reachable which can be seen from the second view of figure 5.

<!-- insert fig 6 -->

Figure 6 is similar to figure 3 but instead of showing the optimal policy for all states, it just shows the optimal policy for the states that are reachable as found in figure 5. This is created by looking at the reachable states found for figure 5, showing only those states at which the optimal policy is to roll. Again, not much is mentioned about this figure in the paper but the one observation given (that the wave tips are not reachable) is consistent to the results that we found.

### 3.4 Winning probability contours (Figure 7)

<!-- Description of fig 7 -->

## 4 Discussion

We were able to verify the main findings of the paper. Importantly, as seen in section $3$, we were able to reproduce all figures that Neller and Presser obtained of their optimal policy. 

### 4.1 Limitations regarding replication

#### 4.1.1 Replication of figures

Our figure $2$ differs from the figure in the paper in the behaviour in the first $5$ iterations, especially for $P_{(1,0,0)}$. In the graph we produced, the estimates for each of the win probabilities are equal to $1$ after the first iteration of the value iteration algorithm. However, in the paper, for $P_{(1,0,0)}$ this win probability is only $0.5$ after the first iteration. This, and other slight differences between the two graphs, could be because of differences between how value iteration is performed in the paper, compared to our replication of the algorithm. Both of the graphs converge to the same values meaning that the probabilities achieved to inform the optimal policy are consistent.

For the figures in section 3.3, the paper does not describe how the authors managed to identify reachable states. We found these states by applying a breadth first search. This was done by starting at $(0,0,0)$ and listing all the states that can be reached directly from this state as determined by the optimal policy to 'hold' or 'roll'. If the optimal action is 'roll', we consider all the possibilities for the value rolled. Then possible scores achievable by player 2 in one turn also need to be considered. Once all these states have been found, any states reachable from these new states are also found and added to the set of reachable states. This is repeated until there are no more new reachable states that can be found. 

Although our figures look similar to those appearing in the paper, it would be helpful to have included specific annotated points or further minor gridlines for verification purposes. 

#### 4.1.2 Percentage win rates

The paper states various win percentages in a competition between various strategies. Our group verified that $P_{(0,0,0)} = 0.5306$. The authors state this implies the probability that an optimal player wins, given that they go first against another optimal player is $53.06\%$. However in our simulations of two optimal players, an empirical estimate of this probability in $100,000$ trials was slightly different at $53.143\%$, with a 95% confidence interval given by $(53.153\%, 53.133\%)$. The authors do not mention conducting a simulation study. 

Further, a comparison is made between a version of a 'hold at $20$' policy and the optimal policy. A hold at 20 policy would entail rolling until the turn total $k=20$ then holding, unless the player is in a winning position, in which case they hold at the amount which first meets or exceeds the $100$ point target. The authors say they use 'the same technique' (pg. 34, Neller and Presser, 2004), to conclude the optimal player, when going first, would win $58.74%$. However it is unclear from the paper to what 'the same technique' refers. There are insufficient details given as to how to modify the equations in earlier sections for the optimal policy versus the hold at $20$ policy, as crucially this breaks any symmetry in the setup. Instead our group attempted to verify the result via simulation, but found different results. In $100,000$ trials for the optimal player going first against the hold at $20$ strategy, we found an estimate of $57.324\%$ win rate with a $95\%$ confidence interval of $(57.334\%, 57.314\%)$. In the same number of trials with order of play reversed the paper claims a win rate of $47.76\%$, and our analysis obtained $49.253\%$, and a 95% confidence interval of $(49.263\%, 49.243\%)$.

As neither of our confidence intervals contained the quoted results of the paper, in further simulation analysis, our group explored the optimal policy against separate strategies of holding at $21$,  $22$ or $23$. We found that the purported win rate of optimal first player against holding at $20$, which was $58.74%$, was closest to that of an optimal first player against the 'hold at $21$' or 'hold at $22$' policies. We calculated point estimates of these win rates to be $58.085\%$ and $58.539\%$ respectively. We noted that the hold at $23$ policy was further away than any of the previous holding policies.

Although it is possible that our competition code is somehow incorrect, or that these frequentist estimates have not converged, it remains unclear how the authors obtained their percentages. If simulation has been used to obtain these results then, these results cannot be considered reproducible. In order for these percentages to be considered reproducible results, the authors should specify the number of trials, and ensure the setting a seed for the random numbers (dice rolls) used in the experiment, and state the accuracy to which the results are calculated or give an indication as to the uncertainty of this estimate (for example via a confidence interval, as we have done above). Finally we conclude that there is evidence that there may be an error in the author's implementation of the 'hold at $20$' policy. 

#### 4.1.3 Convergence criterion 

In the paper a parameter $\Delta$ is introduced to give the tolerance for convergence criterion. However, this threshold is not given at any point in the implementation. We tried $10^{-6}$ through to $10^{-16}$ and found that using the threshold was unable to reproduce the piglet plot in figure 2. Instead, we implemented a maximum number of iterations in our code. However, there still remains an issue as to how to choose the number of iterations for the algorithm.  



### 4.2 Other issues

#### 4.2.1 Computational efficiency claims 

Neller and Cresser describe their method as applying value iteration in stages for subsets of the states. For example they use the fact that there is no transition from $(99,99,0)$ to $(1,1,0)$, and instead limit considerations to those states which could preceed that state such as $(98,99,0)$. The authors describe this as 'partitioning score sums'. It is our understanding that this approach differs from our implementation. We instead have solved all equations iteratively. It remains to be seen if this reduces total number of computations or improves computational efficiency, as the authors claim, as no comparison is made in the paper to a direct approach. 

#### 4.2.2 Value iteration 

The method for finding the optimal policy for the games of piglet and pig are referred to as value iteration throughout the article (pg. 30, Neller and Presser, 2004). Bellman's equation for Value iteration as given in the paper reads:

$$ V(s) \approx \max_{a\in A} \left\{ \sum_{s'\in S:s\mapsto s'} P_{ss'}^{a} [ R_{ss'}^{a} + \gamma V(s')]\right\}.$$ 

It is explained that $\gamma = 1$, that the rewards are $1$ when rolling takes one to a winning state, and that the value of a state $V(s)$ is the probability of eventually winning from that state. However it is not explained to the reader how one encodes the symmetry into the value function that leads from Bellman's equation to equations of the form (1) on page $28$ of the article. Further all theoretical results concerning convergence require $\gamma < 1$, and insufficient justification is given as to why in this case there may be convergence.


## 5 Conclusions and further work

In this work we have conducted a comprehensive reproducibility study of the paper by Neller and Presser on the optimal play of the game pig. In doing so, we have produced a package and repository which makes available the code for the reproduction of all figures in the paper. Further we include the .pckl (pickle) file which contains the optimal policy as we have used, which can be readily compared to policies of other users, or to judge the level of convergence. Although we found that all the figures were reproducible, it was not always clear how to do so from the paper alone.

The terminology of the paper partially aligns with the application of Value Iteration to Markov Decision Processes (Ch 9, Poole and Mackworth 2017). Here the language of discount factors, values and policies are interpreted for the game pig. However, there are aspects of the problem for which there is no straightforward alignment with the MDP literature, such as the value of the 'next' state in the MDP approach being identified through problem-specific symmetry to another part of the state space. It would be interesting to research if there are other problems for which the value of a state is a function, in some generalised MDP approach. 

As discussed in section 4 we found some small discrepancies with the percentage comparisons to other strategies including the hold at $20$ policy, and it is our view that these are not reproducible numbers.

In the course of our study we contacted the authors of the paper, who supplied their original Java code for solving piglet using a backward iteration approach as described earlier. They explained that they would be unable to share further code due to their use at Gettysburg College as assessed teaching materials. In further work we could translate this to Python and compare computational efficiency of both methods to evidence the efficiency claims. 



## References

Neller, Todd W. and Clifton G.M. Presser. (2004) "Optimal Play of the Dice Game Pig," The UMAP Journal 25.1 , 25-47.

Poole, D. L., & Mackworth, A. K. (2017). Artificial Intelligence: Foundations of Computational Agents (Second edition.). Cambridge University Press. https://doi.org/10.1017/9781108164085